In [ ]:
!git clone https://github.com/geohot/tinygrad.git

Cloning into 'tinygrad'...
remote: Enumerating objects: 14566, done.
remote: Counting objects: 100% (1892/1892), done.
remote: Compressing objects: 100% (387/387), done.
remote: Total 14566 (delta 1690), reused 1587 (delta 1505), pack-reused 12674
Receiving objects: 100% (14566/14566), 18.42 MiB | 24.56 MiB/s, done.
Resolving deltas: 100% (10290/10290), done.


In [ ]:
cd tinygrad/

/content/tinygrad


In [ ]:
!python3 -m pip install -e .

Obtaining file:///content/tinygrad
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.2/919.2 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 7.9 MB/s eta 0:00:00
  Running setup.py develop for tinygrad


In [ ]:
import time, sys
import json
import tqdm
import numpy as np
from tinygrad.tensor import Tensor
from tinygrad.nn import optim
import tinygrad.nn as nn
from tinygrad.helpers import flatten
from tinygrad.nn.optim import SGD, Adam
from sklearn.datasets import fetch_openml
from tinygrad.state import safe_save, safe_load, get_state_dict, load_state_dict, torch_load

In [ ]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False, )
print(X.shape, y.shape)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


(70000, 784) (70000,)


In [ ]:
class TinyNetModel:
    def __init__(self):
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = x.reshape((x.shape[0], 1, 28, 28))
        x = self.conv1(x)
        x = x.relu()
        x = x.avg_pool2d(2)    ## Conv1:  (64, 6, 14, 14)

        x = self.conv2(x)
        x = x.relu()
        x = x.avg_pool2d(2)    ## Conv2:  (64, 16, 5, 5)

        x = self.conv3(x)
        x = x.relu()

        x = x.reshape((x.shape[0], -1))  ## Reshape:  (64, 120)

        x = self.fc1(x)  ## FC1:  (64, 84)
        x = x.relu()

        x = self.fc2(x)   ## FC2:  (64, 10)
        return x.log_softmax()

net = TinyNetModel()

In [ ]:
Tensor.training = True

In [ ]:
# from extra.training import sparse_categorical_crossentropy
def cross_entropy(out, Y):
  num_classes = out.shape[-1]
  YY = Y.flatten().astype(np.int32)
  y = np.zeros((YY.shape[0], num_classes), np.float32)
  y[range(y.shape[0]),YY] = -1.0*num_classes
  y = y.reshape(list(Y.shape)+[num_classes])
  y = Tensor(y)
  return out.mul(y).mean()

In [ ]:
opt = Adam([net.conv1.weight, net.conv2.weight, net.conv3.weight], lr=3e-4)

In [ ]:
num_epochs = 2000

running_loss, correct, total = 0.0, 0.0, 0.0
for epoch in range(num_epochs):
    start_time = time.time()

    # Randomly sample a batch
    samp = np.random.randint(0, X.shape[0], size=(64))
    batch = Tensor(X[samp].astype('float32') / 255.0, requires_grad=False)
    # Get the corresponding labels
    labels = y[samp]

    # Forward pass
    out = net.forward(batch)

    # Compute loss
    loss = cross_entropy(out, labels)

    # Zero gradients
    opt.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    opt.step()

    # Calculate accuracy
    pred = np.argmax(out.numpy(), axis=-1)
    labels = [eval(label) for label in labels]

    acc = (pred == labels).mean()
    if epoch % 100 == 0:
      print(f"Time Taken: {time.time()-start_time:.3f}s, Epoch [{epoch+1}/{num_epochs}], Loss: {loss.numpy():.5f}, Accuracy: {acc:.5f}")


Time Taken: 0.360s, Epoch [1/2000], Loss: 2.29161, Accuracy: 0.12500
Time Taken: 0.193s, Epoch [101/2000], Loss: 1.92328, Accuracy: 0.68750
Time Taken: 0.202s, Epoch [201/2000], Loss: 0.71991, Accuracy: 0.85938
Time Taken: 0.200s, Epoch [301/2000], Loss: 0.53938, Accuracy: 0.85938
Time Taken: 0.199s, Epoch [401/2000], Loss: 0.54275, Accuracy: 0.76562
Time Taken: 0.206s, Epoch [501/2000], Loss: 0.36100, Accuracy: 0.92188
Time Taken: 0.193s, Epoch [601/2000], Loss: 0.62642, Accuracy: 0.76562
Time Taken: 0.194s, Epoch [701/2000], Loss: 0.25249, Accuracy: 0.95312
Time Taken: 0.208s, Epoch [801/2000], Loss: 0.49148, Accuracy: 0.84375
Time Taken: 0.208s, Epoch [901/2000], Loss: 0.64140, Accuracy: 0.82812
Time Taken: 0.325s, Epoch [1001/2000], Loss: 0.25148, Accuracy: 0.96875
Time Taken: 0.196s, Epoch [1101/2000], Loss: 0.51045, Accuracy: 0.87500
Time Taken: 0.198s, Epoch [1201/2000], Loss: 0.15379, Accuracy: 0.95312
Time Taken: 0.199s, Epoch [1301/2000], Loss: 0.42848, Accuracy: 0.90625
Time

In [ ]:
# set training flag to false
Tensor.training = False

st = time.perf_counter()
avg_acc = 0
for step in range(1000):
  # random sample a batch
  samp = np.random.randint(0, X.shape[0], size=(64))
  batch = Tensor((X[samp].astype('float32') / 255.0), requires_grad=False)
  # get the corresponding labels
  labels = y[samp]

  # forward pass
  out = net.forward(batch)

  # calculate accuracy
  pred = np.argmax(out.numpy(), axis=-1)

  labels = [eval(label) for label in labels]
  avg_acc += (pred == labels).mean()

print(f"Test Accuracy: {avg_acc / 1000}")
print(f"Time Taken To Test: {time.perf_counter() - st}")

Test Accuracy: 0.926109375
Time Taken To Test: 55.83690177400001


In [ ]:
# first we need the state dict of our model
state_dict = get_state_dict(net)

# then we can just save it to a file
safe_save(state_dict, "/content/model.safetensors")

In [ ]:
# ## weight/bias values into the float-32 format

# def list_to_string_val(flatten_list):
#   return " ".join(str(item) for item in flatten_list)


# def get_float_list(ndim_list):
#     value_list = list(ndim_list.numpy())
#     return value_list


# def get_weight_bias_file(model_state_dict):
#   for key, value in model_state_dict.items():
#     model_value_list = np.array(get_float_list(value)).flatten()
#     list_to_string = list_to_string_val(model_value_list)
#     with open('/content/weight_bias_model.txt','ab') as f:
#         f.write(str.encode(list_to_string))
#         f.write(str.encode("\n"))

# get_weight_bias_file(state_dict)

In [ ]:
# ### weight/bias values into the int format

def float32_to_int(num):
    scaled_num = int(round(num * 1000))
    return scaled_num


def list_to_string_val(flatten_list):
    int_list = [float32_to_int(item) for item in flatten_list]  # Convert float values to integers
    return " ".join(str(item) for item in int_list)


def get_float_list(ndim_list):
    value_list = list(ndim_list.numpy())
    return value_list


def get_weight_bias_file(model_state_dict):
    for key, value in model_state_dict.items():
        model_value_list = np.array(get_float_list(value)).flatten()
        list_to_string = list_to_string_val(model_value_list)
        with open('/content/weight_bias_model_int.txt', 'ab') as f:
            f.write(str.encode(list_to_string))
            f.write(str.encode("\n"))

get_weight_bias_file(state_dict)

In [ ]:
# def list_to_string_val(flatten_list):
#   return " ".join(str(item) for item in flatten_list)

def float32_to_int(num):
    scaled_num = int(round(num * 1000))
    return scaled_num

def list_to_string_val(flatten_list):
    int_list = [float32_to_int(item) for item in flatten_list]  # Convert float values to integers
    return " ".join(str(item) for item in int_list)


def extract_feature_map(x):
  feature_map = x.numpy().flatten()
  feature_map_string_values = list_to_string_val(feature_map)

  with open('/content/feature_map_test_int.txt','ab') as f:
          f.write(str.encode(feature_map_string_values))
          f.write(str.encode("\n"))

In [ ]:
class TinyNetTestModel:
    def __init__(self):
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = x.reshape((x.shape[0], 1, 28, 28))
        extract_feature_map(x) # 0
        x = self.conv1(x)
        extract_feature_map(x) # 1
        x = x.relu()
        extract_feature_map(x) # 2

        x = x.avg_pool2d(2)    ## Conv1:  (1, 6, 14, 14)
        extract_feature_map(x) # 3

        x = self.conv2(x)
        extract_feature_map(x)   # 4
        # print("shape of conv2: ", x.shape)
        x = x.relu()
        extract_feature_map(x) # 5

        x = x.avg_pool2d(2)    ## Conv2:  (1, 16, 5, 5)
        extract_feature_map(x) # 6

        x = self.conv3(x)
        extract_feature_map(x)  # 7
        x = x.relu()
        # print("shape of conv3: ", x.shape)
        extract_feature_map(x) # 8

        x = x.reshape((x.shape[0], -1))  ## Reshape:  (1, 120)
        extract_feature_map(x) # 9

        x = self.fc1(x)  ## FC1:  (1, 84)
        extract_feature_map(x) # 10
        x = x.relu()
        extract_feature_map(x) # 11

        x = self.fc2(x)   ## FC2:  (1, 10)
        extract_feature_map(x) # 12
        return x.log_softmax()

checkpoints = {
    'conv1.weight': state_dict['conv1.weight'],
    'conv1.bias': state_dict['conv1.bias'],
    'conv2.weight': state_dict['conv2.weight'],
    'conv2.bias': state_dict['conv2.bias'],
    'conv3.weight': state_dict['conv3.weight'],
    'conv3.bias': state_dict['conv3.bias'],
    'fc1.weight': state_dict['fc1.weight'],
    'fc1.bias': state_dict['fc1.bias'],
    'fc2.weight': state_dict['fc2.weight'],
    'fc2.bias': state_dict['fc2.bias']
}

model = TinyNetTestModel()
load_state_dict(model, checkpoints)

ram used:  0.28 GB, fc2.bias                                          : 100%|██████████| 10/10 [00:00<00:00, 614.31it/s]

loaded weights in 19.70 ms, 0.28 GB loaded at 14.41 GB/s


## Inference on Unseen Image

In [ ]:
img = [
      0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
      0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
      0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
      0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,84,
      185,159,151,60,36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,222,254,254,254,254,241,198,198,198,198,
      198,198,198,198,170,52,0,0,0,0,0,0,0,0,0,0,0,0,67,114,72,114,163,227,254,225,254,254,254,250,229,254,
      254,140,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,66,14,67,67,67,59,21,236,254,106,0,0,0,0,0,0,0,0,0,0,0,
      0,0,0,0,0,0,0,0,0,0,0,0,0,83,253,209,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22,233,255,83,
      0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,129,254,238,44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
      0,0,0,0,59,249,254,62,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,133,254,187,5,0,0,0,0,0,0,0,0,
      0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,205,248,58,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,126,254,182,
      0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,75,251,240,57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
      0,0,0,19,221,254,166,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,203,254,219,35,0,0,0,0,0,0,0,0,0,
      0,0,0,0,0,0,0,0,0,0,0,0,0,0,38,254,254,77,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31,224,254,115,
      1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,133,254,254,52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
      0,0,0,61,242,254,254,52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121,254,254,219,40,0,0,0,0,0,0,0,
      0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121,254,207,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
      0,0,0,0,0,0,0,0,0,0,0,0,0,0
]

In [ ]:
img = np.array(img)
image = img.astype(np.float32)/255.0
# print(image)
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

preprocessed_image = image.reshape((1, 1, 28, 28))

In [ ]:
Tensor.training = False
input_tensor = Tensor(preprocessed_image)  # Create a tensor from the preprocessed image
output_tensor = model.forward(input_tensor)
predicted_class= np.argmax(output_tensor.numpy(), axis=-1)

np.set_printoptions(formatter={'float_kind':'{:f}'.format})
np.set_printoptions(threshold=sys.maxsize)
print("Predicted class:", predicted_class)

Predicted class: [7]


## Test Accuracy Score

In [ ]:
file_path = "/content/mnist_test.csv"

data = np.loadtxt(file_path, delimiter=",")

X_test = data[:, 1:]
y_test = data[:, 0]

X_test = X_test / 255.0

print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_test shape: (10000, 784)
y_test shape: (10000,)


In [ ]:
import tqdm
avg_acc, correct_pred, false_pred = 0, 0, 0
batch_size = 128

num_batches = int(np.ceil(len(X_test) / batch_size))
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(X_test))

    batch_images = X_test[start_idx:end_idx].reshape((-1, 1, 28, 28))
    input_tensor = Tensor(batch_images)  # Convert the batch of preprocessed images to a tensor

    output_tensor = model.forward(input_tensor)
    predicted_classes = np.argmax(output_tensor.numpy(), axis=-1)

    actual_labels = y_test[start_idx:end_idx].astype(int)

    avg_acc += np.sum(predicted_classes == actual_labels)
    correct_pred += np.sum(predicted_classes == actual_labels)
    false_pred += np.sum(predicted_classes != actual_labels)

avg_acc /= len(X_test)

print("Accuracy of Prediction: ", avg_acc)
print("True Positive Value: ", correct_pred)
print("False Negative Value: ", false_pred)

Accuracy of Prediction:  0.9383
True Positive Value:  9383
False Negative Value:  617
